In [ ]:
import pandas as pd

import importlib, sys

if mod := sys.modules.get("psql_utils.epsql"): importlib.reload(mod)
import psql_utils.epsql as epsql
engine = epsql.Engine()

if mod := sys.modules.get("import_nhgis_shapefiles"): importlib.reload(mod)
from import_nhgis_shapefiles import nhgis_geo_download_year

pd.set_option('display.max_columns', 500)


In [ ]:
for year in [1990, 2000, 2008, 2009, 2010, 2011, 2012, 2013,2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]:
    nhgis_geo_download_year(year)

In [ ]:
gdf = engine.execute_returning_gdf("SELECT * FROM nhgis_geo_wgs84.block_2000_tl2010 where geoid between '420030' and '420033'")
gdf.explore()


In [ ]:
import import_nhgis_shapefiles
import_nhgis_shapefiles.check_and_index_geoid_column(2010, "block", "tl2020", create_index=False)

In [ ]:
df = engine.execute_returning_df("select * from nhgis_geo_wgs84.block_2010_tl2020")

In [ ]:
df

In [ ]:
len(set(df.geoid)), len(df.geoid)

In [ ]:
gdups = df[df.duplicated(subset=["geoid"], keep=False)].sort_values("geoid")
print(len(gdups))
gdups

In [ ]:
dups = df[df.duplicated(keep=False)].sort_values("geoid")
print(len(dups))
dups

In [ ]:
# Show up to 1000 lines of dataframes
pd.set_option('display.max_rows', 1000)

In [ ]:
import shapely


dupcopy = dups.copy()
dupcopy["geom"] = dupcopy["geom"].apply(lambda x: shapely.wkb.loads(x))
dupcopy=gpd.GeoDataFrame(dupcopy, geometry="geom", crs="EPSG:4326")
dupcopy.explore()


In [ ]:
dupcopy.iloc[0].geom

In [ ]:
gpd.GeoDataFrame(, geometry="geom").explore()
# Create a geodataframe from gdups, converting geom column from hex-encoded WKB to shapely geometry


In [ ]:
# for table_name in ["tract_1990_tl2000", "tract_2000_tl2000", "block_1990_tl2000", "block_2000_tl2000", "block_2000_tl2010", "block_2010_tl2010", "block_2010_tl2020", "block_2020_tl2020"]:
#     engine.execute(f"drop table if exists nhgis_geo_wgs84.{table_name}")


In [ ]:
import geopandas as gpd
al=gpd.read_file("nhgis_geo_downloads/block_2010_tl2010/extract_975/shapefiles/nhgis0975_shapefile_tl2010_010_block_2010/AL_block_2010.shp", ignore_geometry=True)
ak=gpd.read_file("nhgis_geo_downloads/block_2010_tl2010/extract_975/shapefiles/nhgis0975_shapefile_tl2010_020_block_2010/AK_block_2010.shp", ignore_geometry=True)
az=gpd.read_file("nhgis_geo_downloads/block_2010_tl2010/extract_975/shapefiles/nhgis0975_shapefile_tl2010_040_block_2010/AZ_block_2010.shp", ignore_geometry=True)

In [ ]:
az["ALAND10"].dtype

In [ ]:
engine.execute("drop table if exists nhgis_geo_wgs84.block_2010_tl2010")

In [ ]:
#engine().execute("ALTER TABLE nhgis_geo_wgs84.tract_2000_tl2008 DROP COLUMN geoid")
#engine().execute_returning_df("select * from nhgis_geo_wgs84.tract_2000_tl2000 where geoid='40210000700';")


In [ ]:
    # def shapefile_path(self, state_fips):
    #     name = self.geography_name(state_fips)
    #     dest = f'{tiger_downloads}/{name}'

    #     if not os.path.exists(dest):
    #         src = self.src(state_fips)

    #         utils.download_file(src, f'{dest}.zip')
    #         utils.unzip_file(f'{dest}.zip')
    #         os.unlink(f'{dest}.zip')
        
    #     shapefile_candidates = glob.glob(f'{dest}/*.shp')
    #     assert len(shapefile_candidates) == 1
    #     return shapefile_candidates[0]

    # def read_shapefile_as_wgs84(self, state_fips: str):
    #     gdf = gpd.read_file(self.shapefile_path(state_fips))
    #     epsql.sanitize_column_names(gdf, inplace=True)
    #     gdf.rename_geometry('geom', inplace=True)
    #     gdf.to_crs(epsg=4326, inplace = True)
    #     return gdf

    # def geography_name(self, state_fips: str):
    #     return f'tl_{self.year}_{state_fips}_{self.level_name}'
    
    # def geography_table_name(self):
    #     return f'tiger_wgs84.tl_{self.year}_{self.level_name}'

    # def src(self, state_fips: str):
    #     src = (
    #         f'https://www2.census.gov/geo/tiger/TIGER{self.year}/{self.download_subdir}/'
    #         f'{self.geography_name(state_fips)}.zip'
    #     )
    #     return src

    # def index_geography_geometries(self, engine):
    #     table_name = self.geography_table_name()
    #     schema = epsql.get_schema(table_name)
    #     with engine().connect() as con:
    #         con.execute(f'CREATE INDEX IF NOT EXISTS {epsql.get_table_name(table_name)}_geom_idx ON {table_name} USING GIST (geom)')
    #         con.execute(f'CREATE INDEX IF NOT EXISTS {epsql.get_table_name(table_name)}_geoid_idx ON {table_name} ({self.geoid_column_name})')



In [ ]:


# #%%

# def add_census_geoids(engine, dest_table, dest_geom_column, year, verbose=False):
#     print(f'Adding census geoids to {dest_table}.{dest_geom_column} from TIGER year {year}')
#     for geography in NhgisGeographyTable.geography_tables(year):
#         census_table = geography.table_name()
#         census_geoid_column = geography.geoid_column_name
#         dest_geoid_column = f'{dest_geom_column}_{geography}_{census_geoid_column}'
#         engine().execute(f'ALTER TABLE {dest_table} DROP COLUMN IF EXISTS {dest_geoid_column}')
#         engine().execute(f'ALTER TABLE {dest_table} ADD COLUMN {dest_geoid_column} TEXT')
#         cmd = f"""
#             UPDATE {dest_table} AS dest
#             SET {dest_geoid_column} = tiger.{census_geoid_column}
#             FROM {census_table} AS tiger
#             WHERE ST_Contains(tiger.geom, dest.{dest_geom_column})"""
#         engine().execute(cmd, verbose=verbose)
#         geoid_count = engine().execute_count(f'SELECT COUNT({dest_geoid_column}) FROM {dest_table}')
#         all_count = engine().execute_count(f'SELECT COUNT(*) FROM {dest_table}')
#         print(f'  Created {dest_table}.{dest_geoid_column}, finding {geoid_count} of {all_count} records')
# # %%
